# Pandas - Spojování více DataFrames

## Úvod do spojování DataFrames

Tento notebook ukazuje různé způsoby spojování DataFrames s vizualizací a podrobným vysvětlením logiky joins.

### Pochopení logiky joins pomocí Vennových diagramů

```
INNER JOIN - Průnik (∩)
┌─────────────────────┐
│ Levý DF     Pravý DF│
│    ┌───┐   ┌───┐    │
│    │ A │   │ C │    │  
│    │   │ ██│██ │    │  ← Pouze společné klíče
│    │ B │ ██│██ │ D  │
│    └───┘   └───┘    │
└─────────────────────┘

LEFT JOIN - Zachová levý 
┌─────────────────────┐
│ Levý DF     Pravý DF│
│  ████████   ┌───┐   │
│  ████████ ██│██ │   │  ← Všechny z levého + průnik
│  ████████ ██│██ │ D │
│  ████████   └───┘   │
└─────────────────────┘

RIGHT JOIN - Zachová pravý
┌─────────────────────┐
│ Levý DF     Pravý DF│
│    ┌───┐ ████████   │
│    │ A │ ████████   │  ← Všechny z pravého + průnik  
│    │   │ ████████   │
│    │ B │ ████████   │
│    └───┘ ████████   │
└─────────────────────┘

OUTER JOIN - Sjednocení (∪)
┌─────────────────────┐
│ Levý DF     Pravý DF│
│  ████████ ████████  │
│  ████████ ████████  │  ← Všechno z obou
│  ████████ ████████  │
│  ████████ ████████  │
└─────────────────────┘
```

### Praktická ukázka na datech

```
LEVÝ DATAFRAME (teploty)       PRAVÝ DATAFRAME (srazky)
┌─────────┬─────────┐          ┌─────────┬─────────┐
│  mesto  │ teplota │          │  mesto  │ srazky  │
├─────────┼─────────┤          ├─────────┼─────────┤
│  praha  │   -2    │          │  praha  │   45    │
│   brno  │   -1    │          │   brno  │   50    │  
│ ostrava │    0    │          │  plzen  │   38    │
└─────────┴─────────┘          └─────────┴─────────┘
                               
SPOLEČNÉ KLÍČE: praha, brno
POUZE V LEVÉM: ostrava  
POUZE V PRAVÉM: plzen
```

## Přehled metod pro spojování

### Metody pro spojování DataFrames
| Metoda | Co ovlivňuje | Popis | Příklad použití |
|--------|--------------|-------|-----------------|
| `pd.concat()` | **Řádky nebo Sloupce** | Spojení více DataFrames vertikálně/horizontálně | `pd.concat([df1, df2])` |
| `df.merge()` | **Řádky podle klíče** | SQL-style joins podle společných sloupců | `df1.merge(df2, on='key')` |
| `df.join()` | **Sloupce podle indexu** | Spojení podle indexu (rychlejší než merge) | `df1.join(df2)` |
| `df.combine_first()` | **Hodnoty** | Doplnění chybějících hodnot z jiného DataFrame | `df1.combine_first(df2)` |
| `df.compare()` | **Rozdíly** | Porovnání dvou DataFrames a zobrazení rozdílů | `df1.compare(df2)` |

### Typy spojení pro merge
| Typ spojení | Parametr | Popis | Analogie |
|-------------|----------|-------|----------|
| **Inner join** | `how='inner'` | Pouze společné klíče | Průnik množin |
| **Left join** | `how='left'` | Všechny řádky z levého DataFrame | Zachová "hlavní" tabulku |
| **Right join** | `how='right'` | Všechny řádky z pravého DataFrame | Zachová "vedlejší" tabulku |
| **Outer join** | `how='outer'` | Všechny řádky z obou DataFrames | Sjednocení množin |

### Klíčové parametry
| Parametr | Metoda | Popis | Výchozí hodnota |
|----------|--------|-------|-----------------|
| `axis` | concat | Osa spojení (0=řádky, 1=sloupce) | `0` |
| `ignore_index` | concat | Vytvoření nového indexu | `False` |
| `on` | merge | Sloupec(e) pro spojení | `None` |
| `left_on/right_on` | merge | Různé názvy klíčů | `None` |
| `how` | merge/join | Typ spojení | `'inner'`/`'left'` |
| `suffixes` | merge/join | Přípony pro duplicitní sloupce | `('_x', '_y')` |
| `result_names` | compare | Názvy výsledných sloupců | `('self', 'other')` |
| `sort` | merge/concat | Řazení výsledků | `False` |

### Další použité metody
| Metoda | Popis | Příklad |
|--------|-------|---------|
| `df.set_index()` | Nastavení sloupce jako index | `df.set_index('mesto')` |
| `df.reset_index()` | Resetování indexu | `df.reset_index()` |
| `df.groupby()` | Seskupení dat | `df.groupby('mesto')` |
| `df.mean()` | Průměr hodnot | `df.mean()` |
| `df.iloc[]` | Přístup podle pozice | `df.iloc[:, 1:]` |
| `df.columns` | Seznam sloupců | `df.columns[0]` |
| `len()` | Počet řádků | `len(df)` |
| `list()` | Převod na seznam | `list(df.index)` |

## Concat - Jednoduché spojování

**Co dělá:** Spojuje DataFrames "lepením" - buď řádky pod sebe, nebo sloupce vedle sebe

### Vizualizace concat transformace

```
VERTIKÁLNÍ CONCAT (axis=0) - řádky pod sebe
DF1                    DF2
┌─────────┬─────────┐  ┌─────────┬─────────┐
│  mesto  │ teplota │  │  mesto  │ teplota │
├─────────┼─────────┤  ├─────────┼─────────┤
│  praha  │   -2    │  │  brno   │   -1    │
│ ostrava │    0    │  │  plzen  │    1    │
└─────────┴─────────┘  └─────────┴─────────┘
                ↓ pd.concat([df1, df2])
        VÝSLEDEK - slepené pod sebe
        ┌─────────┬─────────┐
        │  mesto  │ teplota │
        ├─────────┼─────────┤
        │  praha  │   -2    │ ← z DF1
        │ ostrava │    0    │ ← z DF1
        │   brno  │   -1    │ ← z DF2  
        │  plzen  │    1    │ ← z DF2
        └─────────┴─────────┘

HORIZONTÁLNÍ CONCAT (axis=1) - sloupce vedle sebe  
DF1                    DF2
┌─────────┬─────────┐  ┌─────────┬─────────┐
│  mesto  │ teplota │  │  mesto  │ srazky  │
├─────────┼─────────┤  ├─────────┼─────────┤
│  praha  │   -2    │  │  praha  │   45    │
│   brno  │   -1    │  │   brno  │   50    │
└─────────┴─────────┘  └─────────┴─────────┘
                ↓ pd.concat([df1, df2], axis=1)
            VÝSLEDEK - slepené vedle sebe
            ┌─────────┬─────────┬─────────┬─────────┐
            │  mesto  │ teplota │  mesto  │ srazky  │
            ├─────────┼─────────┼─────────┼─────────┤
            │  praha  │   -2    │  praha  │   45    │
            │   brno  │   -1    │   brno  │   50    │
            └─────────┴─────────┴─────────┴─────────┘
```

### Praktický příklad

In [20]:
import pandas as pd
import numpy as np

# Teploty z různých stanic
stanice_a = pd.DataFrame({
    'mesto': ['praha', 'ostrava'],
    'teplota': [-2, 0]
})

stanice_b = pd.DataFrame({
    'mesto': ['brno', 'plzen'], 
    'teplota': [-1, 1]
})

print("PŘED - Dvě samostatné stanice:")
print("Stanice A:")
print(stanice_a)
print("\nStanice B:")
print(stanice_b)
print()

# Vertikální concat - řádky pod sebe
spojena_data = pd.concat([stanice_a, stanice_b])
print("PO - Vertikální concat (řádky pod sebe):")
print(spojena_data)
print()

# Reset indexu pro lepší přehlednost
spojena_cisty_index = pd.concat([stanice_a, stanice_b], ignore_index=True)
print("S novým indexem:")
print(spojena_cisty_index)

PŘED - Dvě samostatné stanice:
Stanice A:
     mesto  teplota
0    praha       -2
1  ostrava        0

Stanice B:
   mesto  teplota
0   brno       -1
1  plzen        1

PO - Vertikální concat (řádky pod sebe):
     mesto  teplota
0    praha       -2
1  ostrava        0
0     brno       -1
1    plzen        1

S novým indexem:
     mesto  teplota
0    praha       -2
1  ostrava        0
2     brno       -1
3    plzen        1


### Horizontální concat - sloupce vedle sebe

In [21]:
# Data pro horizontální spojení
teploty = pd.DataFrame({
    'mesto': ['praha', 'brno'],
    'teplota': [-2, -1]
})

srazky = pd.DataFrame({
    'mesto': ['praha', 'brno'],
    'srazky': [45, 50]
})

print("PŘED - Dvě tabulky se stejnými indexy:")
print("Teploty:")
print(teploty)
print("\nSrážky:")
print(srazky)
print()

# Horizontální concat - sloupce vedle sebe
horizont_spojena = pd.concat([teploty, srazky], axis=1)
print("PO - Horizontální concat (sloupce vedle sebe):")
print(horizont_spojena)
print()
print("Pozor: duplicitní sloupec 'mesto'!")

# Řešení duplicitních sloupců
horizont_bez_duplikatu = pd.concat([teploty, srazky.iloc[:, 1:]], axis=1)  # vynechá sloupec 0
print("\nBez duplicitního sloupce:")
print(horizont_bez_duplikatu)

PŘED - Dvě tabulky se stejnými indexy:
Teploty:
   mesto  teplota
0  praha       -2
1   brno       -1

Srážky:
   mesto  srazky
0  praha      45
1   brno      50

PO - Horizontální concat (sloupce vedle sebe):
   mesto  teplota  mesto  srazky
0  praha       -2  praha      45
1   brno       -1   brno      50

Pozor: duplicitní sloupec 'mesto'!

Bez duplicitního sloupce:
   mesto  teplota  srazky
0  praha       -2      45
1   brno       -1      50


### Concat s různými indexy

In [22]:
# Data s různými indexy
df1 = pd.DataFrame({
    'a': [1, 2]
}, index=[0, 1])

df2 = pd.DataFrame({
    'b': [3, 4]  
}, index=[1, 2])

print("Data s různými indexy:")
print("DF1:")
print(df1)
print("\nDF2:")
print(df2)
print()

# Horizontální concat s různými indexy
concat_rozne_indexy = pd.concat([df1, df2], axis=1)
print("Horizontální concat s různými indexy:")
print(concat_rozne_indexy)
print("NaN tam, kde chybí index!")

Data s různými indexy:
DF1:
   a
0  1
1  2

DF2:
   b
1  3
2  4

Horizontální concat s různými indexy:
     a    b
0  1.0  NaN
1  2.0  3.0
2  NaN  4.0
NaN tam, kde chybí index!


### Zacházení s NaN hodnotami, duplicity a indexy

In [23]:
print("CONCAT - Zacházení s daty:")
print("="*40)
print("NaN hodnoty: Zachovává všechny NaN")
print("Duplicitní indexy: Zachovává duplicity") 
print("Původní index: Zachovává původní indexy")
print("Duplicitní sloupce: Zachovává všechny sloupce")
print()

# Ukázka zachování duplicitních indexů
df_duplikaty = pd.DataFrame({'a': [1, 2]}, index=[0, 0])  # duplicitní index
df_normal = pd.DataFrame({'b': [3, 4]}, index=[0, 1])

concat_duplikaty = pd.concat([df_duplikaty, df_normal])
print("Concat zachovává duplicitní indexy:")
print(concat_duplikaty)
print(f"Indexy: {list(concat_duplikaty.index)}")

CONCAT - Zacházení s daty:
NaN hodnoty: Zachovává všechny NaN
Duplicitní indexy: Zachovává duplicity
Původní index: Zachovává původní indexy
Duplicitní sloupce: Zachovává všechny sloupce

Concat zachovává duplicitní indexy:
     a    b
0  1.0  NaN
0  2.0  NaN
0  NaN  3.0
1  NaN  4.0
Indexy: [0, 0, 0, 1]


## Merge - SQL-style spojování

**Co dělá:** Spojuje DataFrames podle společných hodnot v klíčových sloupcích (jako SQL JOIN)

### Praktické příklady

#### Inner Join

In [24]:
import pandas as pd
# Základní data
teploty = pd.DataFrame({
    'mesto': ['praha', 'brno', 'ostrava'],
    'teplota': [-2, -1, 0]
})

srazky = pd.DataFrame({
    'mesto': ['praha', 'brno', 'plzen'],
    'srazky': [45, 50, 38]
})

print("PŘED - Dva DataFrames:")
print("Teploty:")
print(teploty)
print("\nSrážky:")
print(srazky)
print()

# Inner join - pouze společné klíče
inner_vysledek = teploty.merge(srazky, on=teploty.columns[0], how='inner')  # sloupec 0
print("PO - Inner join (pouze společné klíče):")
print(inner_vysledek)
print(f"Počet řádků: {len(inner_vysledek)} (původně {len(teploty)} a {len(srazky)})")

PŘED - Dva DataFrames:
Teploty:
     mesto  teplota
0    praha       -2
1     brno       -1
2  ostrava        0

Srážky:
   mesto  srazky
0  praha      45
1   brno      50
2  plzen      38

PO - Inner join (pouze společné klíče):
   mesto  teplota  srazky
0  praha       -2      45
1   brno       -1      50
Počet řádků: 2 (původně 3 a 3)


#### Left Join

In [25]:
# Left join - zachová levý DataFrame
left_vysledek = teploty.merge(srazky, on=teploty.columns[0], how='left')
print("Left join (všechny z levého):")
print(left_vysledek)
print("Ostrava má NaN srážky, protože není v pravém DataFrame")

Left join (všechny z levého):
     mesto  teplota  srazky
0    praha       -2    45.0
1     brno       -1    50.0
2  ostrava        0     NaN
Ostrava má NaN srážky, protože není v pravém DataFrame


#### Right Join

In [26]:
# Right join - zachová pravý DataFrame  
right_vysledek = teploty.merge(srazky, on=teploty.columns[0], how='right')
print("Right join (všechny z pravého):")
print(right_vysledek)
print("Plzeň má NaN teplotu, protože není v levém DataFrame")

Right join (všechny z pravého):
   mesto  teplota  srazky
0  praha     -2.0      45
1   brno     -1.0      50
2  plzen      NaN      38
Plzeň má NaN teplotu, protože není v levém DataFrame


#### Outer Join

In [27]:
# Outer join - všechny kombinace
outer_vysledek = teploty.merge(srazky, on=teploty.columns[0], how='outer')
print("Outer join (všechny z obou):")
print(outer_vysledek)
print("Obsahuje všechna města s NaN tam, kde chybí data")

Outer join (všechny z obou):
     mesto  teplota  srazky
0    praha     -2.0    45.0
1     brno     -1.0    50.0
2  ostrava      0.0     NaN
3    plzen      NaN    38.0
Obsahuje všechna města s NaN tam, kde chybí data


### Merge s různými názvy klíčů

In [28]:
# DataFrames s různými názvy klíčových sloupců
df_cesky = pd.DataFrame({
    'mesto': ['praha', 'brno'],
    'teplota': [-2, -1]
})

df_anglicky = pd.DataFrame({
    'city': ['praha', 'brno'],
    'humidity': [80, 75]
})

print("DataFrames s různými názvy klíčů:")
print("Český:")
print(df_cesky)
print("\nAnglický:")
print(df_anglicky)
print()

# Merge s různými názvy klíčů - použijeme čísla sloupců
vysledek_ruzne_klice = df_cesky.merge(df_anglicky, 
                                     left_on=df_cesky.columns[0],    # sloupec 0 z levého
                                     right_on=df_anglicky.columns[0]) # sloupec 0 z pravého

print("Merge s různými názvy klíčů:")
print(vysledek_ruzne_klice)

DataFrames s různými názvy klíčů:
Český:
   mesto  teplota
0  praha       -2
1   brno       -1

Anglický:
    city  humidity
0  praha        80
1   brno        75

Merge s různými názvy klíčů:
   mesto  teplota   city  humidity
0  praha       -2  praha        80
1   brno       -1   brno        75


### Zacházení s NaN hodnotami, duplicity a indexy

In [29]:
import pandas as pd

print("MERGE - Zacházení s daty:")
print("="*40)

# Test s duplicitními klíči
duplikaty_left = pd.DataFrame({
    'mesto': ['praha', 'praha', 'brno'],  # praha 2x
    'teplota': [-2, -1, 0]
})

duplikaty_right = pd.DataFrame({
    'mesto': ['praha', 'brno'],
    'srazky': [45, 50]
})

print("Duplicitní klíče:")
print("Left:")
print(duplikaty_left)
print("\nRight:")  
print(duplikaty_right)
print()

# Merge s duplicitními klíči vytvoří kartézský součin
vysledek_duplikaty = duplikaty_left.merge(duplikaty_right, on=duplikaty_left.columns[0])
print("Merge s duplicitními klíči (kartézský součin):")
print(vysledek_duplikaty)
print()

print("MERGE vlastnosti:")
print("NaN hodnoty: Vytváří NaN při nematching klíčích")
print("Duplicitní klíče: Kartézský součin (všechny kombinace)")
print("Původní index: Vytváří nový index 0,1,2...")
print("Duplicitní sloupce: Přidává suffixes (_x, _y)")

MERGE - Zacházení s daty:
Duplicitní klíče:
Left:
   mesto  teplota
0  praha       -2
1  praha       -1
2   brno        0

Right:
   mesto  srazky
0  praha      45
1   brno      50

Merge s duplicitními klíči (kartézský součin):
   mesto  teplota  srazky
0  praha       -2      45
1  praha       -1      45
2   brno        0      50

MERGE vlastnosti:
NaN hodnoty: Vytváří NaN při nematching klíčích
Duplicitní klíče: Kartézský součin (všechny kombinace)
Původní index: Vytváří nový index 0,1,2...
Duplicitní sloupce: Přidává suffixes (_x, _y)


## Join - Spojování podle indexu

**Co dělá:** Spojuje DataFrames podle indexu - rychlejší než merge pro jednoduchá spojení

### Praktický příklad

In [30]:
import pandas as pd

# DataFrames s indexy
teploty_indexed = pd.DataFrame({
    'teplota': [-2, -1, 0]
}, index=['praha', 'brno', 'ostrava'])

srazky_indexed = pd.DataFrame({
    'srazky': [45, 50, 38]
}, index=['praha', 'brno', 'plzen'])

print("PŘED - DataFrames s indexy:")
print("Teploty:")
print(teploty_indexed)
print("\nSrážky:")
print(srazky_indexed)
print()

# Join podle indexu
join_vysledek = teploty_indexed.join(srazky_indexed, how='left')
print("PO - Join podle indexu (left):")
print(join_vysledek)
print()

# Různé typy join
print("Inner join:")
print(teploty_indexed.join(srazky_indexed, how='inner'))
print()

print("Outer join:")
print(teploty_indexed.join(srazky_indexed, how='outer'))

PŘED - DataFrames s indexy:
Teploty:
         teplota
praha         -2
brno          -1
ostrava        0

Srážky:
       srazky
praha      45
brno       50
plzen      38

PO - Join podle indexu (left):
         teplota  srazky
praha         -2    45.0
brno          -1    50.0
ostrava        0     NaN

Inner join:
       teplota  srazky
praha       -2      45
brno        -1      50

Outer join:
         teplota  srazky
brno        -1.0    50.0
ostrava      0.0     NaN
plzen        NaN    38.0
praha       -2.0    45.0


### Zacházení s NaN hodnotami, duplicity a indexy

In [31]:
import pandas as pd

print("JOIN - Zacházení s daty:")
print("="*40)

# Test s duplicitními indexy
dup_index_left = pd.DataFrame({
    'teplota': [-2, -1]
}, index=['praha', 'praha'])  # duplicitní index

dup_index_right = pd.DataFrame({
    'srazky': [45]
}, index=['praha'])

print("Duplicitní indexy:")
print("Left:")
print(dup_index_left)
print("\nRight:")
print(dup_index_right)
print()

# Join s duplicitními indexy
try:
    vysledek_dup_join = dup_index_left.join(dup_index_right)
    print("Join s duplicitními indexy:")
    print(vysledek_dup_join)
except Exception as e:
    print(f"Chyba při join s duplicitními indexy: {e}")

print()
print("JOIN vlastnosti:")
print("NaN hodnoty: Vytváří NaN při nematching indexech")
print("Duplicitní indexy: Může způsobit problémy/chyby")
print("Původní index: Zachovává index z levého DataFrame")
print("Duplicitní sloupce: Přidává suffixes (_x, _y)")

JOIN - Zacházení s daty:
Duplicitní indexy:
Left:
       teplota
praha       -2
praha       -1

Right:
       srazky
praha      45

Join s duplicitními indexy:
       teplota  srazky
praha       -2      45
praha       -1      45

JOIN vlastnosti:
NaN hodnoty: Vytváří NaN při nematching indexech
Duplicitní indexy: Může způsobit problémy/chyby
Původní index: Zachovává index z levého DataFrame
Duplicitní sloupce: Přidává suffixes (_x, _y)


### Join vs Merge - kdy použít co

In [32]:
# Porovnání rychlosti a použití
print("POROVNÁNÍ: Join vs Merge")
print("="*40)
print("JOIN:")
print("+ Rychlejší pro spojení podle indexu")
print("+ Jednodušší syntax")
print("- Pouze spojení podle indexu")
print("- Problémy s duplicitními indexy")
print()
print("MERGE:")
print("+ Flexibilnější (podle jakéhokoliv sloupce)")
print("+ Zvládá duplicitní klíče (kartézský součin)")
print("+ Více možností (suffixes, různé klíče)")
print("- Pomalejší")

POROVNÁNÍ: Join vs Merge
JOIN:
+ Rychlejší pro spojení podle indexu
+ Jednodušší syntax
- Pouze spojení podle indexu
- Problémy s duplicitními indexy

MERGE:
+ Flexibilnější (podle jakéhokoliv sloupce)
+ Zvládá duplicitní klíče (kartézský součin)
+ Více možností (suffixes, různé klíče)
- Pomalejší


## Combine_first - Doplňování dat

**Co dělá:** Doplňuje chybějící hodnoty z jednoho DataFrame pomocí druhého

### Vizualizace combine_first transformace

```
PŘED - DataFrames s chybějícími hodnotami
HLAVNÍ (neúplné data)        DOPLŇUJÍCÍ (backup data)
┌─────────┬─────────┐        ┌─────────┬─────────┐
│  mesto  │ teplota │        │  mesto  │ teplota │
├─────────┼─────────┤        ├─────────┼─────────┤
│  praha  │   -2    │        │  praha  │   -1    │ ← ignoruje se
│   brno  │   NaN   │ ◄────  │   brno  │   -1    │ ← použije se
│ ostrava │    0    │        │ ostrava │   NaN   │ ← ignoruje se
└─────────┴─────────┘        └─────────┴─────────┘
              ↓ df1.combine_first(df2)
        VÝSLEDEK - doplněné chybějící hodnoty
        ┌─────────┬─────────┐
        │  mesto  │ teplota │
        ├─────────┼─────────┤
        │  praha  │   -2    │ ← původní hodnota
        │   brno  │   -1    │ ← doplněno z df2
        │ ostrava │    0    │ ← původní hodnota
        └─────────┴─────────┘
```

### Praktický příklad

In [33]:
import pandas as pd

# Data s chybějícími hodnotami
hlavni_data = pd.DataFrame({
    'mesto': ['praha', 'brno', 'ostrava'],
    'teplota': [-2, np.nan, 0],
    'srazky': [45, 50, np.nan]
})

backup_data = pd.DataFrame({
    'mesto': ['praha', 'brno', 'ostrava', 'plzen'],
    'teplota': [-1, -1, np.nan, 1],
    'srazky': [40, np.nan, 35, 38]
})

print("PŘED - Data s chybějícími hodnotami:")
print("Hlavní data:")
print(hlavni_data)
print("\nBackup data:")
print(backup_data)
print()

# Nastavení indexu pro combine_first
hlavni_indexed = hlavni_data.set_index(hlavni_data.columns[0])  # sloupec 0 jako index
backup_indexed = backup_data.set_index(backup_data.columns[0])  # sloupec 0 jako index

# Combine_first - doplnění chybějících hodnot
doplnene_data = hlavni_indexed.combine_first(backup_indexed)
print("PO - Doplněné chybějící hodnoty:")
print(doplnene_data)
print()
print("Brno: teplota doplněna z backup (-1)")
print("Ostrava: srážky doplněny z backup (35)")
print("Plzeň: přidáno kompletně z backup")

PŘED - Data s chybějícími hodnotami:
Hlavní data:
     mesto  teplota  srazky
0    praha     -2.0    45.0
1     brno      NaN    50.0
2  ostrava      0.0     NaN

Backup data:
     mesto  teplota  srazky
0    praha     -1.0    40.0
1     brno     -1.0     NaN
2  ostrava      NaN    35.0
3    plzen      1.0    38.0

PO - Doplněné chybějící hodnoty:
         teplota  srazky
mesto                   
brno        -1.0    50.0
ostrava      0.0    35.0
plzen        1.0    38.0
praha       -2.0    45.0

Brno: teplota doplněna z backup (-1)
Ostrava: srážky doplněny z backup (35)
Plzeň: přidáno kompletně z backup


### Zacházení s NaN hodnotami, duplicity a indexy

In [34]:
import pandas as pd

print("COMBINE_FIRST - Zacházení s daty:")
print("="*40)

# Test s duplicitními indexy
dup_hlavni = pd.DataFrame({
    'hodnota': [1, np.nan]
}, index=['a', 'a'])  # duplicitní index

dup_backup = pd.DataFrame({
    'hodnota': [2, 3]
}, index=['a', 'a'])  # duplicitní index

print("Duplicitní indexy:")
print("Hlavní:")
print(dup_hlavni)
print("\nBackup:")
print(dup_backup)
print()

# Combine_first s duplicitními indexy
vysledek_dup_combine = dup_hlavni.combine_first(dup_backup)
print("Combine_first s duplicitními indexy:")
print(vysledek_dup_combine)
print()

print("COMBINE_FIRST vlastnosti:")
print("NaN hodnoty: Inteligentně doplňuje z druhého DF")
print("Duplicitní indexy: Kombinuje podle pozice")
print("Původní index: Zachovává a rozšiřuje index z levého")
print("Duplicitní sloupce: Kombinuje hodnoty, nepřidává suffixes")

COMBINE_FIRST - Zacházení s daty:
Duplicitní indexy:
Hlavní:
   hodnota
a      1.0
a      NaN

Backup:
   hodnota
a        2
a        3

Combine_first s duplicitními indexy:
   hodnota
a      1.0
a      3.0

COMBINE_FIRST vlastnosti:
NaN hodnoty: Inteligentně doplňuje z druhého DF
Duplicitní indexy: Kombinuje podle pozice
Původní index: Zachovává a rozšiřuje index z levého
Duplicitní sloupce: Kombinuje hodnoty, nepřidává suffixes


## Compare - Porovnání DataFrames

**Co dělá:** Porovnává dva DataFrames a ukazuje rozdíly

### Vizualizace compare transformace

```
PŘED - Dva podobné DataFrames
DF1 (původní)               DF2 (upravené)
┌─────────┬─────────┐       ┌─────────┬─────────┐
│  mesto  │ teplota │       │  mesto  │ teplota │
├─────────┼─────────┤       ├─────────┼─────────┤
│  praha  │   -2    │ ≠     │  praha  │   -1    │ ← ROZDÍL
│   brno  │   -1    │ =     │   brno  │   -1    │ ← STEJNÉ
│ ostrava │    0    │ ≠     │ ostrava │    1    │ ← ROZDÍL
└─────────┴─────────┘       └─────────┴─────────┘
              ↓ df1.compare(df2)
        VÝSLEDEK - pouze rozdílné hodnoty
        ┌─────────────────┬─────────────────┐
        │     teplota     │     teplota     │
        │      self       │     other       │
        ├─────────────────┼─────────────────┤
        │  praha    -2    │  praha    -1    │
        │ ostrava    0    │ ostrava    1    │
        └─────────────────┴─────────────────┘
```

### Praktický příklad

In [35]:
# Původní a upravená data
puvodni_data = pd.DataFrame({
    'mesto': ['praha', 'brno', 'ostrava'],
    'teplota': [-2, -1, 0],
    'srazky': [45, 50, 35]
})

upravena_data = pd.DataFrame({
    'mesto': ['praha', 'brno', 'ostrava'],
    'teplota': [-1, -1, 1],      # praha a ostrava změněny
    'srazky': [45, 55, 35]       # brno změněno
})

print("PŘED - Porovnání dat:")
print("Původní data:")
print(puvodni_data)
print("\nUpravená data:")
print(upravena_data)
print()

# Nastavení indexu pro compare
puvodni_indexed = puvodni_data.set_index(puvodni_data.columns[0])  # sloupec 0 jako index
upravena_indexed = upravena_data.set_index(upravena_data.columns[0])  # sloupec 0 jako index

# Compare - porovnání rozdílů
rozdily = puvodni_indexed.compare(upravena_indexed)
print("PO - Pouze rozdílné hodnoty:")
print(rozdily)
print()

# Compare s vlastními názvy
rozdily_pojmenovane = puvodni_indexed.compare(upravena_indexed, 
                                             result_names=('puvodni', 'nove'))
print("S vlastními názvy:")
print(rozdily_pojmenovane)

PŘED - Porovnání dat:
Původní data:
     mesto  teplota  srazky
0    praha       -2      45
1     brno       -1      50
2  ostrava        0      35

Upravená data:
     mesto  teplota  srazky
0    praha       -1      45
1     brno       -1      55
2  ostrava        1      35

PO - Pouze rozdílné hodnoty:
        teplota       srazky      
           self other   self other
mesto                             
praha      -2.0  -1.0    NaN   NaN
brno        NaN   NaN   50.0  55.0
ostrava     0.0   1.0    NaN   NaN

S vlastními názvy:
        teplota       srazky      
        puvodni nove puvodni  nove
mesto                             
praha      -2.0 -1.0     NaN   NaN
brno        NaN  NaN    50.0  55.0
ostrava     0.0  1.0     NaN   NaN


### Zacházení s NaN hodnotami, duplicity a indexy

In [36]:
import pandas as pd

print("COMPARE - Zacházení s daty:")
print("="*40)

# Test s NaN hodnotami
nan_puvodni = pd.DataFrame({
    'hodnota': [1, np.nan, 3]
}, index=['a', 'b', 'c'])

nan_nove = pd.DataFrame({
    'hodnota': [1, 2, np.nan]
}, index=['a', 'b', 'c'])

print("Data s NaN:")
print("Původní:")
print(nan_puvodni)
print("\nNové:")
print(nan_nove)
print()

# Compare s NaN hodnotami
vysledek_nan_compare = nan_puvodni.compare(nan_nove)
print("Compare s NaN hodnotami:")
print(vysledek_nan_compare)
print()

print("COMPARE vlastnosti:")
print("NaN hodnoty: Považuje NaN vs hodnotu za rozdíl")
print("Duplicitní indexy: Srovnává podle pozice")
print("Původní index: Zachovává původní indexy")
print("Duplicitní sloupce: Vytváří MultiIndex sloupce")

COMPARE - Zacházení s daty:
Data s NaN:
Původní:
   hodnota
a      1.0
b      NaN
c      3.0

Nové:
   hodnota
a      1.0
b      2.0
c      NaN

Compare s NaN hodnotami:
  hodnota      
     self other
b     NaN   2.0
c     3.0   NaN

COMPARE vlastnosti:
NaN hodnoty: Považuje NaN vs hodnotu za rozdíl
Duplicitní indexy: Srovnává podle pozice
Původní index: Zachovává původní indexy
Duplicitní sloupce: Vytváří MultiIndex sloupce


## Praktické tipy

### Kdy použít kterou metodu

| Situace | Doporučená metoda | Důvod |
|---------|------------------|-------|
| Spojení řádků pod sebe | `concat()` | Nejjednodušší pro vertikální spojení |
| Spojení podle společného klíče | `merge()` | Flexibilní, SQL-like logika |
| Spojení podle indexu | `join()` | Rychlejší než merge pro indexy |
| Doplnění chybějících hodnot | `combine_first()` | Specializované na missing data |
| Porovnání rozdílů | `compare()` | Jasně ukáže změny |

